# *to do:*

- put labels on graphs, relabel key
- more cluster analysis (grouping countries based on features)
- validity of ladder score (big assumption that "ladder score" = happiness)

# World Happiness Index
#### Oyun Adilbish, Lane Maitland, Jesse Price, Varun Saini
#### <span style="color:purple"> iXperience - Data Science - Purple </span>.
#### July 22, 2021

# Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Pre-Processing

In [ ]:
# read csv files into data frames
df = pd.read_csv("data/world-happiness-report.csv")
df2021 = pd.read_csv("data/world-happiness-report-2021.csv")

Note that we have two data sets. The first contains data from 2008-2020, and the second contains data from 2021. We wish to combine these into one data frame, so we must examine their columns.

In [ ]:
# examine names of columns for 2008-2020 data frame
list(df.columns)

In [ ]:
# examine names of columns for 2021 data frame
list(df2021.columns)

The column names do not match. We will make the following adjustments:
- add a column to the 2021 data frame to store the year
- drop columns whose data will not be analyzed
- rename the columns so that measurements for the same variable from different data sets will land in a single column
- add a column to the 2005-2020 data frame to store the regional indicator

In renaming the columns, we assume that healthy life expectancy at birth is the same as healthy life expectancy.

In [ ]:
# add column for year on 2021 data frame
df2021['Year'] = 2021

In [ ]:
# drop columns whose data will not be analyzed
df2021 = df2021.drop(columns=['Standard error of ladder score','upperwhisker', 'lowerwhisker',
       'Ladder score in Dystopia',
       'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual'], axis=1)
df = df.drop(columns=['Positive affect','Negative affect'], axis=1)

In [ ]:
# rename columns
df2021 = df2021.rename(columns = {'Logged GDP per capita':'Log GDP per capita'} )
df = df.rename(columns= {'year':'Year', 'Life Ladder':'Ladder score', 'Healthy life expectancy at birth' : 'Healthy life expectancy'})

To add the regional indicator to the 2005-2020 data frame, we merge this data frame with one that consists of two columns from the 2021 data frame. Specifically, we take only the country and region, and we merge on the country.

In [ ]:
# add column for regional indicator on 2005-2020 data frame
df_with_regions = pd.merge(df, df2021[['Country name','Regional indicator']], on = ['Country name'])

We now combine the data frames by using concatination, confirm that we have data from years 2005-2021, and add a column to classify whether the year was pre-COVID or post-COVID. Note that we assume 2005-2019 to be pre-COVID and 2020-2021 to be post-COVID. We then reorder the columns so that columns related to names and dates appear before the others.

In [ ]:
# combine data sets to create one data frame for 2005-2021
df_all = pd.concat([df_with_regions, df2021], ignore_index=True).sort_values(by="Country name", ascending=True)
years_list = list(df_all["Year"].unique())
sorted(years_list)

In [ ]:
# add column for time in terms of COVID
def covid_time(row):
    if row["Year"] < 2020: #distinguishing between pre-covid and post-covid could be improved if we had data by month and if we had data in future
        return "Pre-COVID"  #assuming that the pre-covid is before 2020
    else:
        return "Post-COVID" #assuming that the post-covid is 2020 & 2021

df_all["COVID time"] = df_all.apply(covid_time, axis=1)

In [ ]:
# reorder columns
df_all = df_all[['Country name',
                 'Regional indicator',
                 'Year',
                 'COVID time',
                 'Ladder score',
                 'Log GDP per capita',
                 'Healthy life expectancy',
                 'Social support',
                 'Freedom to make life choices',
                 'Generosity',
                 'Perceptions of corruption']]

In [ ]:
# confirm that data frame has the necessary columns in the intended order
list(df_all.columns)

In [ ]:
# observe the head and tail of the processed data frame
df_all

In [ ]:
# check for missing values
df.isna().sum()

We chose not to remove rows with missing values because a lot of our analysis relies on averages, and we did not want to shorten our data set.

# Sort Countries by Average Ladder Score (2005-2021)

In [ ]:
df_all_ladder = df_all.groupby(['Country name', 'Regional indicator'])['Ladder score'].mean().sort_values(ascending=False)
print("top 15:\n", df_all_ladder.head(15))
print("\nbottom 15:\n", df_all_ladder.tail(15))

We observe that many of the countries in the top/bottom 10 belong to the same region, so we feel that performing an analysis of average ladder score by region will be safe and valid. If the performance of countries in the same region differed greatly, we may wish to proceed with an analysis by country.

# Average Ladder Score by Region

## 2021

In [ ]:
df2021.groupby('Regional indicator')['Ladder score'].mean().sort_values(ascending=False).plot.barh()

plt.xlabel("Ladder score")
plt.title("Average Ladder Score by Region (2021)")
plt.savefig('figures/by_regions_2021.png', dpi=300, bbox_inches='tight')

## 2005-2021

In [ ]:
df_region = df_all.groupby(['Regional indicator', 'Year'])['Ladder score'].mean().to_frame().reset_index()
df_pivot_region = df_region.pivot("Year", "Regional indicator", "Ladder score")

sns.set(style="white", font_scale=1.2)
sns.relplot(data=df_pivot_region, kind="line", height=5, aspect=3, dashes=False)

plt.ylabel("Ladder score") 
plt.title("Average Ladder Score by Region (2005-2021)")
plt.savefig("figures/by_region_all.png", dpi=300, bbox_inches='tight')

## Pre-COVID vs. Post-COVID

For our analysis of the impact of COVID, we chose to only examine 2018-2019 as pre-COVID since we only had two years of data for post-COVID. We believed that the ladder score before 2018 was not representative of a country's performance near the start of the pandemic.

We note that our analysis could be improved if we had obtained data by month, as the effects of the coronavirus began earlier (and persisted later) in some countries than others.

In [ ]:
sns.set(style="white", font_scale=1.2)
covid_plot = sns.catplot(x="Ladder score", y="Regional indicator", hue="COVID time", data=df_all[df_all['Year'] >= 2018], kind="bar", height=5, aspect=3, ci="sd")

plt.title("Average Ladder Score by Region (Pre-COVID vs. Post-COVID)")
plt.savefig('figures/covid.png', dpi=300, bbox_inches='tight')

Observe that the blue and orange bars do not differ much in length, and the error bars overlap greatly. For each region, ladder score did not change drastically as a result of COVID.

# Correlation

We now examine how well the ladder score of a country correlates to the features in the data set. In doing so, we aim to explore whether ladder score is influenced by some factors more than others.

## Correlation Table

In [ ]:
df_all.corr()

## Heat Map

In [ ]:
# remove the year column because its integer value is irrelevent to correlation with ladder score
corrMatrix = df_all.drop("Year", axis =1).corr() 

f,ax=plt.subplots(figsize=(8,8))
hmap = sns.heatmap(corrMatrix, annot=True, linewidth=.5,fmt='.2f',ax=ax, square=True, cmap='RdYlBu_r')
hmap.set_xticklabels(hmap.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title("Heat Map")
plt.savefig('figures/heatmap.png', dpi=300, bbox_inches='tight')

# Clustering

In [ ]:
# review the columns that have numeric values

list(df_all.select_dtypes(include='number').columns)

In [ ]:
from sklearn import mixture

start = df_all.groupby('Country name').mean()

X = start.select_dtypes(include='number')
X = X[['Ladder score', 'Log GDP per capita', 'Healthy life expectancy', 'Social support', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']].copy()

model = mixture.GaussianMixture(n_components=3, covariance_type='full') 
model.fit(X)
y = model.predict(X)
y = y + 1

# remove the year column because its integer value is irrelevent to correlation with ladder score
to_plot = start.copy().reset_index().drop("Year", axis=1)
to_plot.loc[:, 'cluster'] = pd.Series(y)

## Review Countries in Each Cluster

In [ ]:
cluster_sorted = to_plot.sort_values('Country name')

# highest cluster
highest = cluster_sorted.query("`Country name` == 'United States'")['cluster'].iloc[0]

# middle cluster
middle = cluster_sorted.query("`Country name` == 'Russia'")['cluster'].iloc[0]

# lowest cluster
lowest = cluster_sorted.query("`Country name` == 'South Africa'")['cluster'].iloc[0]

print("highest: ", list(cluster_sorted.query('cluster == @highest')['Country name'].unique()))
print("\nmiddle: ", list(cluster_sorted.query('cluster == @middle')['Country name'].unique()))
print("\nlowest: ", list(cluster_sorted.query('cluster == @lowest')['Country name'].unique()))

## Cluster Breakdown by Regional Indicator

In [ ]:
regional = pd.merge(cluster_sorted, df2021[['Country name', 'Regional indicator']], on='Country name')
unstacked = regional.groupby('Regional indicator')['cluster'].value_counts().unstack()
unstacked.rename(columns={highest: 'highest', middle: 'middle', lowest: 'lowest'}, inplace=True)

In [ ]:
ax_h = sns.catplot(data = unstacked.reset_index(), x='highest', y='Regional indicator', kind='bar', aspect=2)
ax_h.set(xlabel='Number of Countries in the Highest Cluster')

In [ ]:
ax_m = sns.catplot(data = unstacked.reset_index(), x='middle', y='Regional indicator', kind='bar', aspect=2)
ax_m.set(xlabel='Number of Countries in the Middle Cluster')

In [ ]:
ax_l = sns.catplot(data = unstacked.reset_index(), x='lowest', y='Regional indicator', kind='bar', aspect=2)
ax_l.set(xlabel='Number of Countries in the Lowest Cluster')

## Pair Plot (Clustered)

In [ ]:
%matplotlib inline
sns.set(style="white", font_scale=1.2)

pair = sns.pairplot(data=to_plot, hue='cluster')
pair.savefig('figures/pairplot_3clusters.png', dpi=300, bbox_inches='tight')

In [ ]:
# define clusters
# choose some plots/relationships to discuss!

## Corruption Analysis Using Clusters

### Ladder score vs. Perceptions of corruption

In [ ]:
ladder_corruption = sns.relplot(data=to_plot, x='Ladder score', y='Perceptions of corruption', hue='cluster')
ladder_corruption.savefig('figures/laddervscorruption.png', dpi=300, bbox_inches='tight')

In [ ]:
gdp_corruption = sns.relplot(data=to_plot, x='Log GDP per capita', y='Perceptions of corruption', hue='cluster')
gdp_corruption.savefig('figures/gdpvscorruption.png', dpi=300, bbox_inches='tight')

### Ladder Score Distribution

In [ ]:
ladder_dist = sns.kdeplot(data=to_plot, shade=True, x='Ladder score', hue='cluster')
ladder_dist.figure.savefig('figures/ladderdistribution.png', dpi=300, bbox_inches='tight')

### Perception of Corruption Distribution

In [ ]:
corruption_dist = sns.kdeplot(data=to_plot, shade=True, x='Perceptions of corruption', hue='cluster')
corruption_dist.figure.savefig('figures/corruptiondistribution.png', dpi=300, bbox_inches='tight')

# Dimensionality Reduction

## 2-Dimensional

In [ ]:
%matplotlib inline
from sklearn.decomposition import PCA
model = PCA(n_components=3)
model.fit(X)
X_2D = model.transform(X)

to_plot['PCA1'] = X_2D[:, 0]
to_plot['PCA2'] = X_2D[:, 1]
to_plot['PCA3'] = X_2D[:, 2]
sns.lmplot(x="PCA1", y="PCA2", hue='cluster', data=to_plot, fit_reg=False);

# Linear Regression

## Different Models for Each Feature

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

feature_list = ["Log GDP per capita", "Healthy life expectancy", "Social support",
                "Freedom to make life choices", "Generosity", "Perceptions of corruption"]

y = to_plot["Ladder score"]

for feature in feature_list:
    model = LinearRegression()
    X = to_plot[[feature]]
    model.fit(X, y)
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true=y, y_pred=y_pred)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = r2_score(y_true=y, y_pred=y_pred)
    print(("{}:\n \t mean squared error: {}\n \t mean absolute error: {}\n \t coefficient of determination: {}")
          .format(feature, mse, mae, r2))

## One Model with All Features

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

model = LinearRegression()
model.fit(X_train, y_train)

coefficients = pd.DataFrame([X.columns,model.coef_]).T
coefficients = coefficients.rename(columns={0: 'Feature', 1: 'Coefficient'})
coefficients["Absolute Value of Coefficient"] = coefficients["Coefficient"].abs()
coefficients

In [ ]:
feature_importance = sns.catplot(data=coefficients.sort_values(by="Absolute Value of Coefficient", ascending=False), x="Absolute Value of Coefficient", y="Feature", kind="bar")

In [ ]:
# model prediction on training data
y_train_pred = model.predict(X_train)

mse = mean_squared_error(y_true=y_train, y_pred=y_train_pred)
mae = mean_absolute_error(y_true=y_train, y_pred=y_train_pred)
r2 = r2_score(y_true=y_train, y_pred=y_train_pred)
print(("mean squared error: {}\nmean absolute error: {}\ncoefficient of determination: {}")
      .format(mse, mae, r2))

In [ ]:
# model prediction on testing data
y_test_pred = model.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_test_pred)
mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
r2 = r2_score(y_true=y_test, y_pred=y_test_pred)
print(("mean squared error: {}\nmean absolute error: {}\ncoefficient of determination: {}")
      .format(mse, mae, r2))

In [ ]:
plt.scatter(y_train, y_train_pred)
plt.xlabel("Ladder Score")
plt.ylabel("Predicted Ladder Score")
plt.title("Predicted Ladder Score vs Ladder Score")
plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

model.fit(X_train, y_train)

In [ ]:
# model prediction on testing data
y_test_pred = model.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_test_pred)
mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
r2 = r2_score(y_true=y_test, y_pred=y_test_pred)
print(("mean squared error: {}\nmean absolute error: {}\ncoefficient of determination: {}")
      .format(mse, mae, r2))

Observe that the mean squared error, mean absolute error, and coefficient of determination are very similar for the second linear regression model and the random forest regressor model.

# Pandas Profiling

In [ ]:
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# conda update -n base -c defaults conda
# !conda install -c conda-forge pandas-profiling -y

# from pandas_profiling import ProfileReport

In [ ]:
# my_report = ProfileReport(df_all, title="World Happiness Index", html={'style':{'full width':True}})
# my_report.to_notebook_iframe()
# my_report.to_file(output_file="World Happiness Index.html")